In [43]:
from qiskit import *
from qiskit.circuit.library import *
from qiskit.providers.aer import AerSimulator
method = "matrix_product_state"
device = "CPU"
backend = AerSimulator(method=method, device=device)
import math

In [44]:
from qrao import QuantumRandomAccessEncoding, QuantumRandomAccessOptimizer, MagicRounding
from qiskit.utils import QuantumInstance
import networkx as nx
from qiskit_optimization.applications import Maxcut
from qiskit.circuit.library import EfficientSU2
from qiskit_aer.primitives import Estimator
from qiskit.algorithms.minimum_eigensolvers import VQE
import matplotlib.pyplot as plt
import multiprocessing

## 1) Input CNF
A 3SAT CNF is of the form $(x_1 \lor x_2 \lor x_3) \land (\neg x_2 \lor x_3 \lor x_4)$ for example

In this notebook, we will assume that the cnf variables ($x_1, x_2, ...$) are given as integers (starting from 0) instead of strings, i.e.
$x_1 \mapsto 0$, $x_2 \mapsto 1$, etc... This is not very intuitive but it makes parsing easier. We might change it later.

- $\neg$ operator: The negation is denoted by an `n` in front of a particular variable $\implies$ Ex: $\neg x_2 \mapsto$ `n3`
- $\lor$ operator: The $\lor$ operator is denoted by a space between operands $\implies$ Ex: $x_1 \lor \neg x_2 \lor x_3 \mapsto$ `0 n1 2` 
- $\land$ operator: The $\land$ operator is denoted by a comma (`,`) between clauses $\implies$ Ex: $(x_1 \lor \neg x_2 \lor x_3) \land (\neg x_1 \lor \neg x_2 \lor \neg x_3) \mapsto$ `0 n1 2,n0 n1 n2`

Therefore, for example, given the following CNF:

$(x_1 \lor x_2 \lor \neg x_3) \land (x_3 \lor \neg x_1 \lor \neg x_6) \land (\neg x_2 \lor x_4 \lor x_5) \land (\neg x_4 \lor \neg x_5 \lor x_6)$

the input CNF to this program will be formulated as

`0 1 n2,2 n0 n5,n1 3 4,n3 n4 5` (as a string)


### Parsing the Input CNF
Given an input CNF, we want to parse is so that
- Instead of having variables `0`, `1`, `2`..., `n0`, `n1`, `n2`, ... we will work with only integers from now on, therefore we will transform the input CNF into a list of integers such that `x` $\mapsto$ `x`, and `nx` $\mapsto$ `x+N` (where `N` is the number of variables)
- Ex: `0 1 n2,2 n0 n5,n1 3 4,n3 n4 5` will be parsed to `parsed_cnf = [0,1,8,2,6,11,7,3,4,9,10,5]`
- We don't need commas or spaces anymore since we know that the input is a 3SAT CNF, so we know the first clause is represented by `(0,1,8)`, the second one by `(2,6,11)`, etc.

The function `parse_cnf()` will ask for two inputs from the user:
- The number of variables (here we count $x$ and $\neg x$ as one variable), so the CNF above has `6` variables.
- The CNF (ex: `0 1 n2,2 n0 n5,n1 3 4,n3 n4 5`)

In [46]:
def parse_cnf_from_str(str_cnf, num_variables):
    """Parses a CNF from a string given as argument

    Parameters
    ----------
    str_cnf: str
        the CNF to parse
    num_variables: int
        the number of variables in the CNF to parse
    Returns
    -------
    list(int)
        the parsed CNF according to the conventions we decide to use
    """
    clauses = str_cnf.replace(",", " ")
    parsed_cnf = clauses.split(" ")
    for i in range(len(parsed_cnf)):
        if parsed_cnf[i].startswith("n"):
            parsed_cnf[i] = str(int(parsed_cnf[i][1:])+num_variables)
    parsed_cnf = list(map(lambda elem: int(elem), parsed_cnf))
    return parsed_cnf, num_variables

## 2) Create a graph from the CNF
Let us transform the CNF into a graph $G$, where
- Each clause maps to a triangle
- The weight of an edge $(i,j)$, $W_{ij}$, is set to $M$ if $j = i+N$, otherwise it is set to the number of triangles that share the edge $(i,j)$.

In [49]:
# Create a graph G from an already-parsed CNF
def create_maxcut_graph(parsed_cnf, num_variables, penalty_factor, add_nodes=False):
    """Creates a weighted graph from a CNF

    Parameters
    ----------
    parsed_cnf : list(int)
        the parsed cnf containing values in [0, 2N-1]
    num_variables : int
        the number of variables used in the CNF (= N)
    penalty_factor: int
        the penalty factor used for assuring consistency

    Returns
    -------
    nx.Graph
        the obtained graph
    """
    
    # All the variables used in the parsed cnf (ranging from 0 to 2N-1)
    cnf_variables = set(parsed_cnf)

    # All the variables used in the original CNF (ranging from 0 to N-1)
    variables = set(range(num_variables))

    # Will contain the weight associated with each edge (i,j)
    edge_weights = {}

    G = nx.Graph()

    # Create the vertices
    # vertices = list(cnf_variables) # correct
    vertices = range(2*num_variables)
    G.add_nodes_from(vertices)
    # Iterate over the parsed CNF per clause (3 elements by 3 elements)
    for i in range(0, len(parsed_cnf), 3):

        # First edge
        # If that edge already exists in the map, add 1 to its value
        if (parsed_cnf[i], parsed_cnf[i+1]) in edge_weights:
            edge_weights[(parsed_cnf[i], parsed_cnf[i+1])] = edge_weights[(parsed_cnf[i], parsed_cnf[i+1])] + 1
        else: # Otherwise, set the weight value to 1
            edge_weights[(parsed_cnf[i], parsed_cnf[i+1])] = 1

        # Second edge
        # If that edge already exists in the map, add 1 to its value
        if (parsed_cnf[i], parsed_cnf[i+2]) in edge_weights:
            edge_weights[(parsed_cnf[i], parsed_cnf[i+2])] = edge_weights[(parsed_cnf[i], parsed_cnf[i+2])] + 1
        else: # Otherwise, set the weight value to 1
            edge_weights[(parsed_cnf[i], parsed_cnf[i+2])] = 1

        # Third edge
        # If that edge already exists in the map, add 1 to its value
        if (parsed_cnf[i+1], parsed_cnf[i+2]) in edge_weights:
            edge_weights[(parsed_cnf[i+1], parsed_cnf[i+2])] = edge_weights[(parsed_cnf[i+1], parsed_cnf[i+2])] + 1
        else: # Otherwise, set the weight value to 1
            edge_weights[(parsed_cnf[i+1], parsed_cnf[i+2])] = 1
    
    # Add all the "clause edges" to the graph after calculating all the weights needed
    for (u,v) in edge_weights.keys():
        G.add_edge(u, v, weight=edge_weights[(u,v)], color="b")


    # Add all the "constraint" edges to the graph, with a weight of penalty_factor each
    for var in variables:
        # Check if both x and not(x) exist in the CNF (otherwise we don't need to add the penalty term)
        if var in cnf_variables and var + num_variables in cnf_variables:
            G.add_edge(var, var+num_variables, weight=penalty_factor, color="r")

    return G

## 3) Theoretical Cut Value

### 3.1) NAE-3SAT Minimization Problem
Given a CNF composed of C clauses and with penalty factor M:
If the values $\{w_1, ..., w_{2N}\}$ satisfy the 3-SAT CNF with the NAE constraint, then (proven):
$\begin{equation}
\min\limits_{w_i}\sum\limits_{j = 1}^{C}(w_1^{(j)}w_2^{(j)} + w_1^{(j)}w_3^{(j)} + w_2^{(j)}w_3^{(j)}) + M\sum\limits_{i\in neg\_var} w_iw_{i+N} = -C-M\lvert \texttt{neg\_var}\rvert
\end{equation}$
where $w_k^{(j)}$ is variable $k$ in clause $j$, and $\texttt{neg\_var}$ is the set of variable indices $i$ such that both $w_i$ and $w_{i+N}$ appear in the parsed CNF. 

The minimization problem can be transformed into the following:
$\begin{equation}
\min\limits_{x \in \{-1,1\}^{|V|}} \left\{\sum\limits_{(i,j) \in E}W_{ij}x_ix_j\right\}
\end{equation}$
where $W_{ij}$ is the weight of edge $(i, j)$ from the previously obtained graph (that weight is equal to the number of triangles that share that edge if
$j \neq i + n$, and $M$ otherwise)

### 3.2) Link Between NAE-3SAT and Weighted Max-Cut
The weighted Max-Cut problem is the following:
$\begin{equation}
        \max\limits_{x \in \{-1,1\}^{|V|}} \left\{\frac{1}{2}\sum\limits_{(i,j) \in E} W_{ij}(1-x_ix_j)\right\} = \max\limits_{x \in \{-1,1\}^{|V|}} \left\{\frac{1}{2}\sum\limits_{(i,j)\in E}W_{ij} - \frac{1}{2}\sum\limits_{(i,j) \in E} W_{ij}x_ix_j\right\}
\end{equation}$
where $x_k \in \{-1, 1\}$
Since we want to maximize this, we would like to minimize the second sum (because of the $-$ sign). However this second sum was computed before and is equal to $-C-M\lvert\texttt{neg\_var}\rvert$ if $x$ satisfies the CNF with the NAE constraint. 

Therefore, if $x$ satisfies the CNF with NAE constraint, then the theoretical Max-Cut value is:
$\begin{equation}
\text{MaxCutValue}_{theory} = \frac{1}{2}\sum\limits_{(i,j)\in E}W_{ij}+\frac{1}{2}(C+M|neg\_var|)
\end{equation}$

In the next cell we compute this value from the original CNF and the (equivalent) graph.

In [50]:
# Get the theoretical Max-Cut value according to the previous formula
def get_theoretical_maxcut_value(G, parsed_cnf, num_variables, M):
    """Compute the theoretical max-cut value
    
    Parameters
    ----------
    G : nx.Graph
        the graph whose max-cut to compute
    parsed_cnf : list(int)
        the parsed cnf containing values in [0, 2N-1]
    num_variables: int
        the number of variables used in the CNF (= N)
    M : int
        the penalty factor for consistency

    Returns
    -------
    float
        the theoretical max-cut value
    """
    # cnf_contribution is the second part of the MaxCutValue_theory above
    cut_value = 0
    for u,v in G.edges:
        cut_value += G[u][v]["weight"]/2
        
    for i in range(num_variables):
        if i in parsed_cnf and i + num_variables in parsed_cnf:
            cut_value += M/2
    cut_value += len(parsed_cnf)/6
    return cut_value


## 4) Compute the Max-Cut value Experimentally
We can choose to either solve only the relaxed problem (`should_round=False`) or the relaxed + rounding version (`should_round=True`).

In [51]:
from qiskit.algorithms.optimizers import COBYLA
def compute_max_cut_qrao(G, rounding_scheme, intermediate_results, ansatz_reps, should_round=False):
    """Compute the relaxed max-cut (or rounded max-cut if should_round=True) using the protoype-qrao library

    Parameters
    ----------
    G : nx.Graph
        the graph whose max-cut to compute
    rounding_scheme: RoundingScheme
        the rounding technique to use (ignored when should_round=False)
    intermediate_results: list(float)
        the intermediate energy values computed from the eigensolver 
    should_round: bool
        if False, we do not perform rounding, otherwise we perform rounding  

    Returns
    -------
    Either a float (non-rounded, relaxed max-cut value) or a list[int] containing the solution to the 3SAT-NAE problem
    """

    maxcut = Maxcut(G)
    problem = maxcut.to_quadratic_program()
    encoding = QuantumRandomAccessEncoding(3) # Use (3,1)-QRAC
    encoding.encode(problem)
    ansatz = EfficientSU2(encoding.num_qubits, su2_gates=["rx", "y"], reps=ansatz_reps, entanglement="linear")
    threads = multiprocessing.cpu_count()

    # results will be either a number (if we solve the relaxed version), or 
    # an array of values in {0,1} (if we perform rounding)
    results = None 
    vqe = VQE(
        Estimator(        
            approximation=True,
            abelian_grouping=False,
            backend_options={"method": method, "device": device, "max_parallel_experiments": threads},
        ),
        ansatz,
        COBYLA(maxiter=1000),
        # for now let's have this callback
        callback= lambda c,p,value,m: intermediate_results.append(problem.objective.sense.value * (encoding.offset + value))
    )
    # Create the qrao instance
    qrao = QuantumRandomAccessOptimizer(encoding=encoding, min_eigen_solver=vqe, rounding_scheme=rounding_scheme)

    if not should_round:
        # We just need to solve the relaxed problem
        eigensolver_result, _ = qrao.solve_relaxed()
        # Relaxed MaxCut Value (computed from the gotten minimimum eigenvalue)
        results = problem.objective.sense.value * (encoding.offset + eigensolver_result.eigenvalue.real)
            

    else:
        # Otherwise we solve the relaxed problem but also perform rounding at the end
        rounded_results = qrao.solve()
        results = rounded_results.x # variable assignments
    return results

## 5) Calculate the Error (Only Done After Rounding)
To calculate the error, we will look at the results (`x_values`) which are the values (0 or 1) we assign to the variables, and check two things:
- NAE-SAT errors: If a clause is not NAE-SAT, then we add 1 to the NAE-SAT error
- Consistency errors: If a variable is the same as its negation, we add 1 to the consistency error

In [52]:
def calculate_error(parsed_cnf, num_variables, result):
    """Calculate the NAE-3SAT error

    Parameters
    ----------
    parsed_cnf: list(int)
        the initial parsed CNF
    num_variables: int
        the number of variables used (without distinguishing x and not x)
    result: list(int)
        the resulting variable assignments

    Returns
    int, int
        the NAE-SAT error and the consistency error, respectively
    """
    
    nae_sat_error = 0
    consistency_error = 0
    # Iterate over the clauses of the CNF
    for i in range(0, len(parsed_cnf), 3):
        v1 = result[parsed_cnf[i]]
        v2 = result[parsed_cnf[i+1]]
        v3 = result[parsed_cnf[i+2]]
        # if clause is satisfied but not SAT-NAE, or if the clause is not satisfied, we add 1 to nae_sat_error
        if v1 + v2 + v3 == 3 or v1 + v2 + v3 == 0:
            nae_sat_error += 1
    
    for i in range(num_variables):
        # if a variable is the same as its negation, we add 1 to consistency_error
        if  i + num_variables < len(result) and result[i] == result[i+num_variables]:
            consistency_error += 1
    return nae_sat_error, consistency_error

In [53]:
def filter_nonconsistent_cnfs(cnfs):
    """Filters out the nonconsistent CNFs from a list of CNFs, to make sure QRAO can run them without mapping issues

    Parameters
    ----------
    cnfs: list(str)
        the list of CNFs to filter
    
    Returns
    -------
    list(str)
        the resulting filtered list
    """
    
    filtered = []
    for i in range(len(cnfs)):
        used_variables = set()
        num_vars, cnf = cnfs[i].replace("\n", "").split(" -- ")
        num_vars = int(num_vars)
        parsed_cnf, num_vars = parse_cnf_from_str(cnf, num_vars)
        for j in range(len(parsed_cnf)):
            used_variables.add(parsed_cnf[j])
        num_vars_ok = True
        for j in range(num_vars):
            if j not in used_variables or j + num_vars not in used_variables:
                num_vars_ok = False
        if num_vars_ok:
            filtered.append((cnf, num_vars))
    return filtered

In [54]:
def compute_min_M_value(parsed_cnf):
    """Calculates the minimal M value required to ensure consistency

    Parameters
    ----------
    parsed_cnf: list(int)
        the input CNF

    Returns
    -------
    int
        the resulting penalty factor M
    """
    
    pair_appearances = {}
    for i in range(0, len(parsed_cnf), 3):
        v_1 = parsed_cnf[i]
        v_2 = parsed_cnf[i+1]
        v_3 = parsed_cnf[i+2]
    pair_appearances[(v_1,v_2)] = 1 if (v_1, v_2) not in pair_appearances else pair_appearances[(v_1, v_2)] + 1
    pair_appearances[(v_1,v_3)] = 1 if (v_1, v_3) not in pair_appearances else pair_appearances[(v_1, v_3)] + 1
    pair_appearances[(v_2,v_3)] = 1 if (v_2, v_3) not in pair_appearances else pair_appearances[(v_2, v_3)] + 1
    max_appearance = 0
    for key in pair_appearances:
        if pair_appearances[key] > max_appearance:
            max_appearance = pair_appearances[key]
    return max_appearance + 1

In [55]:
def compute_var_clause_success_rate_nae3sat_cnfs(filename):
    """Reads NAE-3SAT CNFs from a file and runs QRAO on them, and outputs success/failure rates per (num_variables, num_clauses) CNFs

    Parameters
    ----------
    filename: str
        the name of the file containing NAE-3SAT CNFs
    """
    
    num_success = 0
    cnfs = None
    rounding_scheme = MagicRounding(QuantumInstance(backend=backend, shots=1024))
    with open(filename, "r") as sat_file:
        cnfs = sat_file.readlines()
    filtered_cnfs = filter_nonconsistent_cnfs(cnfs)
    num_runs = len(filtered_cnfs)
    vars_clauses_successes = {}
    vars_clauses_runs = {}
    current_run_number = 0
    for cnf_line in filtered_cnfs:
        current_run_number += 1
        (cnf, num_vars) = cnf_line
        num_vars = int(num_vars)
        parsed_cnf, num_vars = parse_cnf_from_str(cnf, num_vars)
        percentage = "%.2f" % float(100*current_run_number/num_runs) + "%"
        M = compute_min_M_value(parsed_cnf)     
        G = create_maxcut_graph(parsed_cnf, num_vars, M)
        intermediate_results = []

        x_values = compute_max_cut_qrao(G, rounding_scheme, intermediate_results, int(len(parsed_cnf)/3), should_round=True)
        naesat_error, consistency_error = calculate_error(parsed_cnf, num_vars, x_values)
        num_clauses = cnf.count(",") + 1
        if naesat_error + consistency_error == 0:
            if (num_vars, num_clauses) not in vars_clauses_successes:
                vars_clauses_successes[(num_vars, num_clauses)] = 1
            else:
                vars_clauses_successes[(num_vars, num_clauses)] += 1
            num_success += 1
        else:
            if (num_vars, num_clauses) not in vars_clauses_successes:
                    vars_clauses_successes[(num_vars, num_clauses)] = 0
        
        if (num_vars, num_clauses) not in vars_clauses_runs:
            vars_clauses_runs[(num_vars, num_clauses)] = 1
        else:
            vars_clauses_runs[(num_vars, num_clauses)] += 1
        success_rate = "%.2f" % float(100*num_success/current_run_number) + "%"
        print(f"Run {current_run_number}/{num_runs} | {percentage} | success_rate = {success_rate}", end="\r")

    total_success_rate = "%.2f" % float(100*num_success/num_runs) + "%" 
    print(f"Total Success Rate: {num_success}/{num_runs} ({total_success_rate})")
    for (num_vars, num_clauses) in vars_clauses_runs:
        v_c_success_rate = "%.2f" % float(100*vars_clauses_successes[(num_vars, num_clauses)]/vars_clauses_runs[(num_vars, num_clauses)]) + "%"
        num_v_c_successes = vars_clauses_successes[(num_vars, num_clauses)]
        num_v_c_runs = vars_clauses_runs[(num_vars, num_clauses)]
        print(f"For {num_vars} variables on {num_clauses} clauses: Success Rate = {v_c_success_rate} ({num_v_c_successes}/{num_v_c_runs})")

In [ ]:
# Run on NAE-3SAT CNFs
compute_var_clause_success_rate_nae3sat_cnfs("nae3sat_cnfs.txt")

In [56]:
def predict_result_for_nae3sat_cnfs(nae_filename, runs_per_cnf):
    """Reads NAE-3SAT CNFs from a file one by one, runs QRAO on them, then calculates and outputs the success rate per CNF.
    
    Parameters
    ----------
    nae_filename: str
        the file to read the NAE-3SAT CNFs from
    runs_per_cnf: int
        the number of runs per CNF
    """

    rounding_scheme = MagicRounding(quantum_instance=QuantumInstance(backend=backend, shots=2000))
    intermediate_results = []
    nae_file = open(nae_filename, "r")
    nae_cnfs = list(map(lambda elem: elem.replace("\n", ""), nae_file.readlines()))
    nae_file.close()
    num_nae_cnfs = len(nae_cnfs)

    for cnf_index in range(len(nae_cnfs)):
        [num_vars_str, cnf] = nae_cnfs[cnf_index].split(" -- ")
        num_vars = int(num_vars_str)
        if num_vars >= 9:
            parsed_cnf, _ = parse_cnf_from_str(cnf, num_vars)
            num_clauses = int(len(parsed_cnf)/3)
            M = compute_min_M_value(parsed_cnf)
            print(f"M = {M}")
            G = create_maxcut_graph(parsed_cnf, num_vars, M)
            individual_success_rate = 0
            for i in range(runs_per_cnf):
                x_values = compute_max_cut_qrao(G, rounding_scheme, intermediate_results, ansatz_reps=max(num_clauses, 10), should_round=True)
                satnae_error, consistency_error = calculate_error(parsed_cnf, num_vars, x_values)
                if satnae_error + consistency_error == 0:
                    individual_success_rate += 1

            print(f"Success Rate For ({num_vars}, {num_clauses}): {100*individual_success_rate/runs_per_cnf}")
            print(f"Progress: {cnf_index+1}/{num_nae_cnfs}")

In [ ]:
# Run on NAE-3SAT CNFs with 10 runs per instance
predict_result_for_nae3sat_cnfs("nae3sat_cnfs.txt", 10)

In [16]:
def classify(nae3sat_filename, nonnae3sat_filename):
    """Classifies CNFs into NAE-3SAT or non-NAE-3SAT based on the relaxed energy value, and outputs the confusion matrix

    Parameters
    ----------
    nae3sat_filename: str
        the name of the file containing NAE-3SAT CNFs
    nonnae3sat_filename: str
        the name of the file containing non-NAE-3SAT CNFs
    """
    
    rounding_scheme = MagicRounding(quantum_instance=QuantumInstance(backend=backend, shots=2000))
    intermediate_results = []
    nae_file = open(nae3sat_filename, "r")
    nae_cnfs = list(map(lambda elem: elem.replace("\n", ""), nae_file.readlines()))
    nae_file.close()
    num_nae_cnfs = len(nae_cnfs)
    nonnae_file = open(nonnae3sat_filename, "r")
    nonnae_cnfs = list(map(lambda elem: elem.replace("\n", ""), nonnae_file.readlines()))
    nonnae_file.close()
    num_nonnae_cnfs  = len(nonnae_cnfs)
    sat_but_predicted_unsat = 0
    sat_and_predicted_sat = 0
    unsat_but_predicted_sat = 0
    unsat_and_predicted_unsat = 0
    for cnf_index in range(len(nae_cnfs)):
        [num_vars_str, cnf] = nae_cnfs[cnf_index].split(" -- ")
        num_vars = int(num_vars_str)
        parsed_cnf, _ = parse_cnf_from_str(cnf, num_vars)
        num_clauses = int(len(parsed_cnf)/3)
        M = compute_min_M_value(parsed_cnf)
        G = create_maxcut_graph(parsed_cnf, num_vars, M)
        theoretical_maxcut_value = get_theoretical_maxcut_value(G, parsed_cnf, num_vars, M)
        inclination = 0
        for _ in range(1):
            relaxed_maxcut_value = compute_max_cut_qrao(G, rounding_scheme, intermediate_results, ansatz_reps=num_vars + num_clauses, should_round=False)
            if theoretical_maxcut_value <= relaxed_maxcut_value:
                inclination += 1
            else:
                inclination -= 1
        sat_and_predicted_sat = sat_and_predicted_sat + 1 if inclination > 0 else sat_and_predicted_sat
        sat_but_predicted_unsat = sat_but_predicted_unsat + 1 if inclination < 0 else sat_but_predicted_unsat
        progress_percentage = "%.2f" % float(100*(cnf_index+1)/num_nae_cnfs) + "%"
        print(f"Progress (SAT): {progress_percentage}", end="\r")
    print()
    print("--- STARTING UNSAT ---")
    for cnf_index in range(len(nonnae_cnfs)):
        [num_vars_str, cnf] = nonnae_cnfs[cnf_index].split(" -- ")
        num_vars = int(num_vars_str)
        parsed_cnf, _ = parse_cnf_from_str(cnf, num_vars)
        num_clauses = int(len(parsed_cnf)/3)
        M = compute_min_M_value(parsed_cnf)
        G = create_maxcut_graph(parsed_cnf, num_vars, M)
        inclination = 0
        for _ in range(1):
            theoretical_maxcut_value = get_theoretical_maxcut_value(G, parsed_cnf, num_vars, M)
            relaxed_maxcut_value = compute_max_cut_qrao(G, rounding_scheme, intermediate_results, ansatz_reps=num_vars + num_clauses, should_round=False)
            if theoretical_maxcut_value > relaxed_maxcut_value:
                inclination += 1
            else:
                inclination -= 1
        unsat_and_predicted_unsat = unsat_and_predicted_unsat + 1 if inclination > 0 else unsat_and_predicted_unsat
        unsat_but_predicted_sat = unsat_but_predicted_sat + 1 if inclination < 0 else unsat_but_predicted_sat
        progress_percentage = "%.2f" % float(100*(cnf_index+1)/num_nonnae_cnfs) + "%"
        print(f"Progress (SAT): {progress_percentage}", end="\r")
    print()
    print()
    print(f"For {num_nae_cnfs} CNFs: predicted {sat_and_predicted_sat} as SAT and {sat_but_predicted_unsat} as UNSAT")
    print(f"For {num_nonnae_cnfs} CNFs: predicted {unsat_and_predicted_unsat} as UNSAT and {unsat_but_predicted_sat} as SAT")

In [ ]:
# Classify NAE-3SAT and non-NAE-3SAT CNFs
classify("nae3sat_cnfs_nonreduced.txt", "non-nae3sat_cnfs_nonreduced.txt")